# Practicumopdrachten Week 2.3 (eerste kans)

## Dependencies importeren

Kopieer in het codeblok hieronder van het vorige practicum de import-code voor de dependencies die het vaakst worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
Zet eventuele warnings uit.

In [1]:

import warnings
warnings.simplefilter('ignore')
import pandas as pd
import sqlite3
import numpy as np

Zet het bestand go_sales.sqlite in hetzelfde mapje als dit notebook.

## Databasetabellen inlezen

Kopieer in het codeblok hieronder van het vorige practicum de code om een connectie met het bestand go_sales.sqlite te maken.

In [2]:
go_sales_con = sqlite3.connect('Great_Outdoors_Data_SQLite\go_sales.sqlite')

Lees van de ingelezen go_sales-database te volgende tabellen in met behulp van "SELECT * FROM *tabel*".
- product
- product_type
- product_line
- sales_staff
- sales_branch
- retailer_site
- country
- order_header
- order_details
- target
- returned_item
- return_reason

In [3]:
queries = {
    'product': 'SELECT * FROM product',
    'product_type': 'SELECT * FROM product_type',
    'product_line': 'SELECT * FROM product_line',
    'sales_staff': 'SELECT * FROM sales_staff',
    'sales_branch': 'SELECT * FROM sales_branch',
    'retailer_site': 'SELECT * FROM retailer_site',
    'country': 'SELECT * FROM country',
    'order_header': 'SELECT * FROM order_header',
    'order_details': 'SELECT * FROM order_details',
    'target': 'SELECT * FROM SALES_TARGETData',
    'returned_item': 'SELECT * FROM returned_item',
    'return_reason': 'SELECT * FROM return_reason'
}


dataframes = {}

# Lees elke tabel in een DataFrame
for table_name, query in queries.items():
    dataframes[table_name] = pd.read_sql_query(query, go_sales_con)

#als je ik elk tabel als een dataframe/ variabele wil behandelen of aanroepen moet ik dit uitvoeren.
for table_name, df in dataframes.items():
    globals()[table_name] = df
    
dataframes

{'product':     PRODUCT_NUMBER INTRODUCTION_DATE PRODUCT_TYPE_CODE PRODUCTION_COST MARGIN  \
 0                1         15-2-2011                 1               4    .33   
 1               10         15-2-2011                 1              10     .4   
 2              100         15-2-2011                17               3     .5   
 3              101        15-12-2019                18          305.54    .43   
 4              102        10-12-2019                18          380.95    .51   
 ..             ...               ...               ...             ...    ...   
 110             95         15-2-2011                16               3     .5   
 111             96         15-2-2011                17           16.43    .28   
 112             97          5-3-2013                17              25    .28   
 113             98         15-2-2011                17               3     .5   
 114             99         15-2-2011                17               2     .6   
 
   

Let op! Voor alle onderstaande opdrachten mag je <u>alleen Python</u> gebruiken, <u>geen SQL!</u>

## Selecties op één tabel zonder functies

Geef een overzicht met daarin de producten en hun productiekosten waarvan de productiekosten lager dan 100 dollar en hoger dan 50 dollar ligt. (2 kolommen, 23 rijen)

In [4]:
# maak productie kost float omdat het nu als string gezien word.
product['PRODUCTION_COST'] = product['PRODUCTION_COST'].astype(float)

Productie_kost = (product['PRODUCTION_COST'] < 100) & (product['PRODUCTION_COST'] > 50)

resultaat = product.loc[(Productie_kost), ['PRODUCT_NAME', 'PRODUCTION_COST']]
resultaat

,PRODUCT_NAME,PRODUCTION_COST
13,Blue Steel Putter,60.00
14,Blue Steel Max Putter,81.80
17,Course Pro Golf Bag,80.00
24,Hibernator Lite,60.00
25,Hibernator,86.00
28,Hibernator Self - Inflating Mat,78.55
31,Hibernator Camp Cot,65.33
32,Canyon Mule Climber Backpack,62.50
56,Granite Climbing Helmet,52.86
58,Husky Harness Extreme,55.00


Geef een overzicht met daarin de producten en hun marge waarvan de marge lager dan 20 % of hoger dan 60 % ligt. (2 kolommen, 7 rijen) 

In [5]:
#een margin wordt eerst als object gezien, dus deze moet eerst omgezet worden naar float

product['MARGIN'] = product['MARGIN'].astype(float)
Marge = (product['MARGIN'] < .20) | (product['MARGIN'] > .60)

resultaat = product.loc[(Marge), ['PRODUCT_NAME', 'MARGIN']]
resultaat

,PRODUCT_NAME,MARGIN
29,Hibernator Pad,0.17
32,Canyon Mule Climber Backpack,0.17
100,BugShield Natural,0.67
101,BugShield Spray,0.67
102,BugShield Lotion Lite,0.70
103,BugShield Lotion,0.63
105,BugShield Extreme,0.63


Geef een overzicht met daarin de landen waar met francs wordt betaald. Sorteer de uitkomst op land.  (1 kolom, 3 rijen)

In [6]:
francs = country['CURRENCY_NAME'] == 'francs'

resultaat = country.loc[(francs), ['COUNTRY']].sort_values(by = 'COUNTRY')
resultaat
#sorteren doe je door daarna .sort_values te doen.

,COUNTRY
7,Belgium
0,France
19,Switzerland


Geef een overzicht met daarin de verschillende introductiedatums waarop producten met meer dan 50% marge worden geïntroduceerd (1 kolom, 7 rijen) 

In [7]:
product['MARGIN'] = product['MARGIN'].astype(float)
Datums = product['MARGIN'] > .50

resultaat = product.loc[(Datums), ['INTRODUCTION_DATE']].drop_duplicates('INTRODUCTION_DATE')   
#bestaande code geeft veel duplicates dus word drop_duplicates aangeroepen
resultaat

,INTRODUCTION_DATE
4,10-12-2019
6,18-12-2019
7,27-12-2019
9,13-1-2020
14,15-12-2019
79,26-10-2016
100,15-2-2011


Geef een overzicht met daarin het eerste adres en de stad van verkoopafdelingen waarvan zowel het tweede adres als de regio bekend is (2 kolommen, 7 rijen)

In [8]:
Adress = (sales_branch['ADDRESS2'].notna()) & (sales_branch['REGION'].notna())
resultaat = sales_branch.loc[(Adress), ['ADDRESS1', 'CITY']]
resultaat

,ADDRESS1,CITY
10,Prol. Paseo de la Reforma No. 51,Distrito Federal
11,202-2-3 Hyakunincho,Tokyo
12,543-225 Asahi,Osaka City
13,2315 Queen's Ave,Melbourne
15,"Avenida Paulista, 333",São Paulo
21,3 Albany Court,Birmingham
27,Singelgravenplein 4,Amsterdam


Geef een overzicht met daarin de landen waar met dollars (dollars of new dollar) wordt betaald. Sorteer de uitkomst op land. (1 kolom, 4 rijen) 

In [9]:
Dollar = (country['CURRENCY_NAME'] == 'dollars') | (country['CURRENCY_NAME'] == 'new dollar')
resultaat = country.loc[(Dollar), ['COUNTRY']].sort_values(by = 'COUNTRY')
resultaat

,COUNTRY
6,Australia
15,Canada
3,Taiwan
14,United States


Geef een overzicht met daarin beide adressen en de stad van vestigingen van klanten waarvan de postcode begint met een ‘D’ (van duitsland). Filter op vestigingen die een tweede adres hebben. (3 kolommen, 2 rijen) 

In [10]:
Vestiging = (retailer_site['POSTAL_ZONE'].str[0] == 'D')

resultaat = retailer_site.loc[(Vestiging) & retailer_site['ADDRESS2'].notna(), ['ADDRESS1', 'ADDRESS2', 'CITY']]
resultaat

,ADDRESS1,ADDRESS2,CITY
205,Röntgenstraße 90,3. Tür rechts,Frankfurt
351,Grubesallee 141,4. Stock,Hamburg


## Selecties op één tabel met functies

Geef het totaal aantal producten dat is teruggebracht (1 waarde) 

In [11]:
returned_item['RETURN_QUANTITY'] = returned_item['RETURN_QUANTITY'].astype(float)
totaal = returned_item['RETURN_QUANTITY'].sum()
print(totaal)

#14306.0

14306.0


Geef het aantal regio’s waarin verkoopafdelingen gevestigd zijn. (1 waarde)

In [12]:
Regio = sales_branch['REGION'].drop_duplicates().count()
print(Regio)

#Beide kunnen gebruikt worden onder meer "Geschikt"

aantal_regios = sales_branch['REGION'].nunique()
print(aantal_regios)




15
15


Maak 3 variabelen:
- Een met de laagste
- Een met de hoogste
- Een met de gemiddelde (afgerond op 2 decimalen)

marge van producten (3 kolommen, 1 rij) 

In [13]:
laagsteMargin = product['MARGIN'].min()
hoogsteMargin = product['MARGIN'].max()
gemMargin = product['MARGIN'].mean()

df = {
    'laagste' : [laagsteMargin],
    'hoogste' : [hoogsteMargin],
    'gem' : [gemMargin]
}

Resultaat = pd.DataFrame(df)
Resultaat

#onderstaande regel kan ook .to_fram om om te zetten naar dataframe en .T om te transponeren (rijen worden kolomen, kolomen worden rijen)
resultaat = product['MARGIN'].agg(['min', 'max', 'mean']).to_frame().T
resultaat


,min,max,mean
MARGIN,0.17,0.7,0.396609


Geef het aantal vestigingen van klanten waarvan het 2e adres niet bekend is (1 waarde)

In [14]:
aantal = retailer_site['ADDRESS2'].isna().sum()
print(aantal)

#286

286


Geef de gemiddelde kostprijs van de verkochte producten waarop korting (unit_sale_price < unit_price) is verleend (1 waarde) 

In [15]:

order_details['UNIT_SALE_PRICE'] = order_details['UNIT_SALE_PRICE'].astype(float)
order_details['UNIT_PRICE'] = order_details['UNIT_PRICE'].astype(float)
order_details['UNIT_COST'] = order_details['UNIT_COST'].astype(float)


#de loc is de functie .mean ook ?

gem = order_details.loc[order_details['UNIT_SALE_PRICE'] < order_details['UNIT_PRICE'], 'UNIT_COST'].mean()
#gem = (order_details[order_details['UNIT_SALE_PRICE'] < order_details['UNIT_PRICE']]['UNIT_COST']).mean()

print(gem)
#125.42305809556548


125.42305809556548


Geef een overzicht met daarin het aantal medewerkers per medewerkersfunctie (2 kolommen, 7 rijen) 

In [16]:
sales_staff.groupby('POSITION_EN', as_index = False)['SALES_STAFF_CODE'].count().rename(columns = {'POSITION_EN' : 'FUNCTIE', 'SALES_STAFF_CODE' : 'AANTAL'})

,FUNCTIE,AANTAL
0,Branch Manager,19
1,District Manager,4
2,General Manager,1
3,Level 1 Sales Representative,13
4,Level 2 Sales Representative,39
5,Level 3 Sales Representative,24
6,Regional Manager,2


Geef een overzicht met daarin per telefoonnummer het aantal medewerkers dat op dat telefoonnummer bereikbaar is. Toon alleen de telefoonnummer waarop meer dan 4 medewerkers bereikbaar zijn. (2 kolommen, 10 rijen) 

In [17]:

resultaat = sales_staff.groupby('WORK_PHONE', as_index=False)['SALES_STAFF_CODE'].count()

#zo of met .query achter.count
meer_dan_4 = resultaat[resultaat['SALES_STAFF_CODE'] > 4]

print(meer_dan_4)

             WORK_PHONE  SALES_STAFF_CODE
4     +(44) 121 3505267                 5
10  +(61) 03 2982  4242                 5
11  +31 (0)20 692 93 94                 5
13    +33 1 68 94 52 20                 5
18     1 (206) 292-0012                 5
19     1 (305) 557-4810                 6
20     1 (310) 281-5722                 5
21     1 (403) 232-5986                 5
22     1 (416) 493-5595                 5
23     1 (617) 268-6754                 6


## Selecties op meerdere tabellen zonder functies

Geef een overzicht met daarin het eerste adres en de stad van vestigingen van klanten uit ‘Netherlands’ (2 kolommen, 20 rijen) 

In [18]:

merged = pd.merge(retailer_site, country, on='COUNTRY_CODE')

nederlandse  = merged[merged['COUNTRY'] == 'Netherlands']

resultaat = nederlandse[['ADDRESS1', 'CITY']]

resultaat



,ADDRESS1,CITY
164,Beets 36,Beets
165,Perestraat 1,Deventer
166,Laan van Meerdervoort 966,Den Haag
167,Kalfjeslaan 47,Amstelveen
168,Startbaan 11,Amstelveen
169,Noorderspoorsingel 28,Groningen
170,Hogehilweg 13,Amsterdam
171,Spoorstraat 43,Varsseveld
173,Westhavenweg 8,Amsterdam
174,Westzeedijk 4,Muiden


Geef een overzicht met daarin de productnamen die tot het producttype ‘Eyewear’ behoren. (1 kolom, 5 rijen) 

In [19]:
merge = pd.merge(product, product_type, on = 'PRODUCT_TYPE_CODE')

Eyewear = merge[merge['PRODUCT_TYPE_EN'] == 'Eyewear']

resultaat = Eyewear[['PRODUCT_NAME']]

resultaat

,PRODUCT_NAME
80,Polar Sun
81,Polar Ice
83,Polar Sports
84,Polar Wave
85,Polar Extreme


Geef een overzicht met daarin alle unieke eerste adressen van klantvestigingen en de voornaam en achternaam van de verkopers die ‘Branch Manager’ zijn en aan deze vestigingen hebben verkocht (3 kolommen, 1 rij) 

In [20]:
merge2 = pd.merge(retailer_site, order_header, on= 'RETAILER_SITE_CODE', how = 'inner')

final_merge = pd.merge(merge2, sales_staff, on = 'SALES_STAFF_CODE', how = 'inner' )

overzicht = final_merge[final_merge['POSITION_EN'] == 'Branch Manager']

resultaat = overzicht[['ADDRESS1', 'FIRST_NAME', 'LAST_NAME']].drop_duplicates()
resultaat


#is drop_duplicates een functie ? dus fout ?


,ADDRESS1,FIRST_NAME,LAST_NAME
3058,"Alameda Santos, 9876",Bayard,Lopes


Geef een overzicht met daarin van de verkopers hun functie en indien zij iets hebben verkocht de datum waarop de verkoop heeft plaatsgevonden. Laat alleen de verschillende namen van de posities zien van de verkopers die het woord ‘Manager’ in hun positienaam hebben staan. (2 kolommen, 9 rijen) 

In [21]:
merge3 = pd.merge(sales_staff, order_header, on= 'SALES_STAFF_CODE')

Namen = merge3[merge3['POSITION_EN'].str.contains('Manager')]

resultaat = Namen[['POSITION_EN', 'ORDER_DATE']]
resultaat
#Alleen 5 rijen ?

,POSITION_EN,ORDER_DATE
5300,Branch Manager,2022-09-09
5301,Branch Manager,2022-10-31
5302,Branch Manager,2022-12-13
5303,Branch Manager,2022-03-13
5304,Branch Manager,2022-01-09


Geef een overzicht met daarin de verschillende namen van producten en bijbehorende namen van producttypen van de producten waarvoor ooit meer dan 750 stuks tegelijk verkocht zijn. (2 kolommen, 9 rijen) 

In [22]:
merge4 = pd.merge(product, product_type, on= 'PRODUCT_TYPE_CODE')

fmerge4 = pd.merge(merge4, order_details, on= 'PRODUCT_NUMBER')

Check4 = fmerge4[fmerge4['QUANTITY'] > 750]

result = Check4[['PRODUCT_NAME', 'PRODUCT_TYPE_EN']].drop_duplicates()
result

,PRODUCT_NAME,PRODUCT_TYPE_EN
8616,Star Peg,Tents
15975,Firefly 2,Lanterns
37903,BugShield Spray,Insect Repellents
38313,BugShield Lotion Lite,Insect Repellents
38770,BugShield Lotion,Insect Repellents
39618,BugShield Extreme,Insect Repellents
40754,Sun Shelter 15,Sunscreen
41106,Sun Shelter 30,Sunscreen
41681,Sun Shield,Sunscreen


Geef een overzicht met daarin de productnamen waarvan ooit meer dan 40% korting is verleend. De formule voor korting is: (unit_price - unit_sale_price) / unit_price (1 kolom, 8 rijen) 

In [23]:
merge5 = pd.merge(product, order_details, on= 'PRODUCT_NUMBER')

merge5['UNIT_PRICE'] = merge5['UNIT_PRICE'].astype(float)
merge5['UNIT_SALE_PRICE'] = merge5['UNIT_SALE_PRICE'].astype(float)

merge5['Korting'] = (merge5['UNIT_PRICE'] -  merge5['UNIT_SALE_PRICE']) /  merge5['UNIT_PRICE']

check5 =  merge5[ merge5['Korting'] > 0.4]

result = check5[['PRODUCT_NAME']].drop_duplicates()
result




,PRODUCT_NAME
2765,Hailstorm Titanium Woods Set
37446,BugShield Natural
37967,BugShield Spray
38306,BugShield Lotion Lite
38772,BugShield Lotion
39561,BugShield Extreme
40799,Sun Shelter 15
41194,Sun Shelter 30


Geef een overzicht met daarin de retourreden van producten waarvan ooit meer dan 90% van de aangeschafte hoeveelheid is teruggebracht (return_quantity/quantity). (1 kolom, 3 rijen) 

In [24]:
merge6 = pd.merge(order_details, returned_item, on = 'ORDER_DETAIL_CODE')
fmerge6 = pd.merge(merge6, return_reason, on = 'RETURN_REASON_CODE')

fmerge6['RETURN_QUANTITY'] = fmerge6['RETURN_QUANTITY'].astype(float)
fmerge6['QUANTITY'] = fmerge6['QUANTITY'].astype(float)

fmerge6['Terug'] = fmerge6['RETURN_QUANTITY'] / fmerge6['QUANTITY']

check6 = fmerge6[fmerge6['Terug'] > 0.9] 

result6 = check6[['RETURN_DESCRIPTION_EN']].drop_duplicates()
result6



,RETURN_DESCRIPTION_EN
0,Wrong product shipped
1,Unsatisfactory product
15,Wrong product ordered


## Selecties op meerdere tabellen met functies

Geef een overzicht met daarin per producttype het aantal producten die tot dat producttype behoren. (2 kolommen, 21 rijen) 

In [25]:
merge7 = pd.merge(product, product_type, on = 'PRODUCT_TYPE_CODE')

fmerge7 = pd.merge(merge7, order_details, on = 'PRODUCT_NUMBER')

resultaat = fmerge7.groupby('PRODUCT_TYPE_EN', as_index = False)['QUANTITY'].sum()
resultaat
#sum() niet count()

,PRODUCT_TYPE_EN,QUANTITY
0,Binoculars,43330
1,Climbing Accessories,142150
2,Cooking Gear,198676
3,Eyewear,53510
4,First Aid,72348
5,Golf Accessories,47988
6,Insect Repellents,270074
7,Irons,14244
8,Knives,134134
9,Lanterns,345096


Geef een overzicht met daarin per land het aantal vestigingen van klanten die zich in dat land bevinden. (2 kolommen, 21 rijen) 

In [26]:
merge8 = pd.merge(retailer_site, country, on = 'COUNTRY_CODE')

resultaat = merge8.groupby('COUNTRY', as_index = False)['RETAILER_SITE_CODE'].count()
resultaat

#kan of retailer_site_code of retailer_code

,COUNTRY,RETAILER_SITE_CODE
0,Australia,10
1,Austria,10
2,Belgium,10
3,Brazil,1
4,Canada,31
5,China,10
6,Denmark,6
7,Finland,5
8,France,29
9,Germany,32


Geef een overzicht met daarin van de producten behorend tot het producttype ‘Cooking Gear’ per productnaam de totaal verkochte hoeveelheid en de gemiddelde verkoopprijs. Sorteer de uitkomst op totaal verkochte hoeveelheid. (4 kolommen, 10 rijen) 

In [27]:
merge9 = pd.merge(product, product_type, on= 'PRODUCT_TYPE_CODE')
fmerge9 = pd.merge(merge9, order_details, on= 'PRODUCT_NUMBER')

check9 = fmerge9[fmerge9['PRODUCT_TYPE_EN'] == 'Cooking Gear']

resultaat = check9.groupby(['PRODUCT_NAME', 'PRODUCT_TYPE_EN'], as_index = False).agg({'QUANTITY' : 'sum', 'UNIT_SALE_PRICE' : 'mean'}).sort_values(by = 'QUANTITY').rename(columns = {'QUANTITY' : 'TOTAAL', 'UNIT_SALE_PRICE' : '~SALE_PRICE'})
resultaat

,PRODUCT_NAME,PRODUCT_TYPE_EN,TOTAAL,~SALE_PRICE
3,TrailChef Deluxe Cook Set,Cooking Gear,3744,120.303667
5,TrailChef Kettle,Cooking Gear,9900,12.130783
7,TrailChef Single Flame,Cooking Gear,10614,63.572397
1,TrailChef Cook Set,Cooking Gear,12700,51.204554
4,TrailChef Double Flame,Cooking Gear,14510,131.348013
8,TrailChef Utensils,Cooking Gear,15928,17.132397
6,TrailChef Kitchen Kit,Cooking Gear,25964,22.210198
0,TrailChef Canteen,Cooking Gear,31712,11.930119
2,TrailChef Cup,Cooking Gear,31834,6.945976
9,TrailChef Water Bag,Cooking Gear,41770,5.999266


Geef een overzicht met daarin van de verkopers behorend tot de verkoopafdeling uit ‘Amsterdam’ de voornaam en achternaam van de verkoper en zijn laagste en hoogste verkooptarget van 2006. (4 kolommen, 4 rijen) 

In [28]:
merge10 = pd.merge(sales_branch, sales_staff, on = 'SALES_BRANCH_CODE')

merge11 = pd.merge(merge10, target, on = 'SALES_STAFF_CODE')

check11 = merge11[(merge11['CITY'] == 'Amsterdam') & (merge11['SALES_YEAR'] == '2006')]

check11['SALES_TARGET'] = check11['SALES_TARGET'].astype(float)

resultaat = check11.groupby(['FIRST_NAME', 'LAST_NAME'], as_index = False).agg(MIN_SALES_TARGET=('SALES_TARGET', 'min'), MAX_SALES_TARGET=('SALES_TARGET', 'max'))
resultaat


#sales_year heeft geen 2006 ? dus ik snap het niet


,FIRST_NAME,LAST_NAME,MIN_SALES_TARGET,MAX_SALES_TARGET


Geef een overzicht met daarin per land de naam van het land, de naam van de stad waar de verkoopafdeling is gevestigd (noem de kolomnaam in het overzicht ‘verkoper’) en het aantal steden waar zich klanten bevinden in dat land (noem de kolomnaam in het overzicht ‘klanten’) (3 kolommen, 29 rijen) 

In [29]:
merge12 = pd.merge(sales_branch, country, on = 'COUNTRY_CODE')

merge12 = pd.merge(merge12, retailer_site, on= 'COUNTRY_CODE')


resultaat = merge12.groupby(['COUNTRY', 'CITY_x'], as_index = False)['RETAILER_SITE_CODE'].count().rename(columns = {'CITY_x' : 'verkoper', 'RETAILER_SITE_CODE' : 'klanten'})
resultaat

,COUNTRY,verkoper,klanten
0,Australia,Melbourne,10
1,Austria,Wien,10
2,Belgium,Heverlee,10
3,Brazil,São Paulo,1
4,Canada,Calgary,31
5,Canada,Toronto,31
6,China,Shanghai,10
7,Finland,Kuopio,5
8,France,Lyon,29
9,France,Paris,29


## Pythonvertalingen van SUBSELECT en UNION met o.a. for-loops

Geef een overzicht met daarin de voornaam en de achternaam van de medewerkers die nog nooit wat hebben verkocht (2 kolommen, 25 rijen) 

In [30]:
medewerkers_zonder_sales = []

for index, row in sales_staff.iterrows():
    staff_code = row['SALES_STAFF_CODE']
    
    if staff_code not in target['SALES_STAFF_CODE'].unique():
        medewerkers_zonder_sales.append((row['FIRST_NAME'], row['LAST_NAME']))

resultaat = pd.DataFrame(medewerkers_zonder_sales, columns=['Voornaam', 'Achternaam'])

resultaat

,Voornaam,Achternaam
0,Walter,Taylor
1,Penelope,Tamrine
2,Polly,Linton
3,Gracy,Gellens
4,Else,Mörike
5,Frank,Fuchs
6,Fritz,Hirsch
7,Jörg,Kunze
8,Maria,Iacobucci
9,Kick,Kalkman


Geef een overzicht met daarin het aantal producten waarvan de marge lager is dan de gemiddelde marge van alle producten samen. Geef in het overzicht tevens aan wat de gemiddelde marge is van dit aantal producten waarvan de marge lager dan de gemiddelde marge van alle producten samen is. (1 kolom, 2 rijen) 

In [31]:
gemiddelde_marge = product['MARGIN'].mean()

producten_met_lagere_marge = product[product['MARGIN'] < gemiddelde_marge]


aantal_producten_met_lagere_marge = len(producten_met_lagere_marge)
gemiddelde_marge_lagere_producten = producten_met_lagere_marge['MARGIN'].mean()

overzicht = pd.DataFrame({
    'Aantal producten met lagere marge dan gemiddelde': [aantal_producten_met_lagere_marge],
    'Gemiddelde marge van deze producten': [gemiddelde_marge_lagere_producten]
})

overzicht

#2 kolomen 1 rij ?

,Aantal producten met lagere marge dan gemiddelde,Gemiddelde marge van deze producten
0,59,0.292203


    Geef een overzicht met daarin de namen van de producten die voor meer dan 500 (verkoopprijs) zijn verkocht maar nooit zijn teruggebracht. (1 kolom, 13 rijen) 

In [32]:

producten_zonder_retour = order_details[~order_details['ORDER_DETAIL_CODE'].isin(returned_item['ORDER_DETAIL_CODE'])]

verkochte_producten_groter_500 = producten_zonder_retour[producten_zonder_retour['UNIT_SALE_PRICE'].astype(float) > 500]

resultaat = verkochte_producten_groter_500.merge(product, on='PRODUCT_NUMBER', how='inner')

overzicht = resultaat[['PRODUCT_NAME']].drop_duplicates()
overzicht


,PRODUCT_NAME
0,Hailstorm Steel Irons
3,Hailstorm Titanium Irons
8,Lady Hailstorm Steel Irons
11,Lady Hailstorm Titanium Irons
14,Hailstorm Titanium Woods Set
18,Hailstorm Steel Woods Set
22,Lady Hailstorm Titanium Woods Set
25,Lady Hailstorm Steel Woods Set
28,Star Dome
32,Star Gazer 2


Geef een overzicht met daarin per (achternaam van) medewerker of hij/zij manager is of niet, door deze informatie toe te voegen als extra 'Ja/Nee'-kolom.<br>
Hint: gebruik een for-loop waarin je o.a. bepaalt of het woord 'Manager' in de functie (position_en) staat. (2 kolommen, 102 rijen).

In [33]:
sales_staff['Manager'] = ''

for index, row in sales_staff.iterrows():
    Huidige_position = sales_staff.at[index, 'POSITION_EN']

    if 'Manager' in Huidige_position:
        sales_staff.at[index, 'Manager'] = 'Ja'
    else:
        sales_staff.at[index, 'Manager'] = 'Nee'
    
Result = sales_staff[['FIRST_NAME', 'Manager']]
Result

,FIRST_NAME,Manager
0,Tuomas,Nee
1,Chang-ho,Nee
2,Jung-ho,Nee
3,Kwei-Ping,Nee
4,Chin-Tsai,Nee
...,...,...
97,Tomás,Nee
98,Bayard,Ja
99,Viviam,Nee
100,Eduardo,Nee


Met de onderstaande code laat je Python het huidige jaar uitrekenen.

In [34]:
from datetime import date
date.today().year

2024

Met de onderstaande code selecteer je op een bepaald jaartal uit een datum.

In [35]:
from datetime import datetime

date_str = '16-8-2013'
date_format = '%d-%m-%Y'
date_obj = datetime.strptime(date_str, date_format)

date_obj.year

2013

Geef met behulp van bovenstaande hulpcode een overzicht met daarin op basis van het aantal jaar dat iemand in dienst is of een medewerker ‘kort in dienst’ (minder dan 25 jaar in dienst) of een ‘lang in dienst’ (groter gelijk dan 12 jaar in dienst) is. Geef daarbij per medewerker in een aparte kolom zowel ‘kort in dienst’ als ‘lang in dienst’ aan. Gebruik (wederom) een for-loop.<br>
(2 kolommen, 102 rijen) 

In [37]:

sales_staff['In dienst'] = ''
#dit doen anders kan ik niet selecteren

for index, row in sales_staff.iterrows():
    huidige_datum_hired = sales_staff.at[index, 'DATE_HIRED']

    date_format = '%d-%m-%Y'
    jaar_hired = datetime.strptime(huidige_datum_hired, date_format)
    aantal_jaar = (date.today().year - jaar_hired.year)

    if (aantal_jaar < 12):
        sales_staff.at[index, 'In dienst'] = 'kort in dienst'
    elif (aantal_jaar >= 25):
        sales_staff.at[index, 'In dienst'] = 'lang in dienst' 


overzicht = sales_staff[['FIRST_NAME', 'In dienst']]
overzicht

#een medewerker kan niet minder dan 25 jaar kort in dienst zijn en meer dan 12 jaar 'lang' in dienst


,FIRST_NAME,In dienst
0,Tuomas,kort in dienst
1,Chang-ho,kort in dienst
2,Jung-ho,kort in dienst
3,Kwei-Ping,kort in dienst
4,Chin-Tsai,
...,...,...
97,Tomás,kort in dienst
98,Bayard,kort in dienst
99,Viviam,kort in dienst
100,Eduardo,kort in dienst
